# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [15]:
import pandas as pd
import numpy as np
from scipy import stats
import math

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


1.What test is appropriate for this problem? Does CLT apply?

    The two sample test (white sample versus black sample) that compares the proportions of candidates who got the call is appropriate since the two samples are independent of each other and among themselves.

In [6]:
len(data[data.race=='b'])

2435

In [7]:
len(data[data.race=='w'])

2435

This sample is large enough where we can apply CLT here (the mean of all the samples is the same as the mean of the population)

2.What are the null and alternate hypotheses?

Null: Race has no significant factor in determining callbacks. We'll know if this is true if the proportions of black callbacks and white callbacks are equal

Alternate: Race DOES have an impact in determining callbacks.

3.Compute margin of error, confidence interval, and p-value

In [9]:
black_prop = sum(data[data.race=='b'].call)/len(data[data.race=='b'])
print(black_prop)

0.064476386037


In [10]:
white_prop = sum(data[data.race=='w'].call)/len(data[data.race=='w'])
print(white_prop)

0.0965092402464


So already we see that the proportions are off. Lets see what the variances are

In [11]:
black_var = black_prop*(1 - black_prop)/(len(data[data.race=='b']))
print(black_var)

2.47717378565e-05


In [13]:
white_var = white_prop*(1 - white_prop)/(len(data[data.race=='b']))
print(white_var)

3.5809119833e-05


So the variance of the difference in the black_prop and white_prop variables are 2.47717378565e-05 and 3.5809119833e-05 respectively. This indicates that the white sample set is more spread out than the black sample set.

Next, we should find the total variance in the population, not just the two samples individually.

In [14]:
total_var = white_var + black_var
print(total_var)

6.05808576895e-05


From here, we can use this number to find the standard deviation of the dataset.

In [16]:
total_stddev = math.sqrt(total_var)
print(total_stddev)

0.0077833705866767544


In [18]:
conf_margin = 1.95 * total_stddev
conf_margin

0.015177572644019671

So the margin of error for a 95% confidence interval is +/- ~0.01526

In [19]:
sample_diff = white_prop - black_prop
sample_diff

0.032032854209445585

In [20]:
sample_diff + conf_margin

0.047210426853465257

In [21]:
sample_diff - conf_margin

0.016855281565425914

Since we're looking at the difference between the two samples, I got the difference between their proportions and used that to determine the confidence interval (1.68% and 4.73%)

Now assuming our null hypothesis is true, each of the proportions SHOULD be:

In [22]:
Null_prop = (sum(data[data.race=='b'].call) + sum(data[data.race=='w'].call))/len(data)
Null_prop

0.080492813141683772

The standard error of this is:

In [23]:
Null_SD = math.sqrt(2 * Null_prop * (1 - Null_prop)/len(data[data.race=='w']))
Null_SD

0.007796894036170457

And now to find the p-value

In [26]:
x =sample_diff/Null_SD
x

4.1084121524343464

In [27]:
1 - stats.norm(0,1).cdf(x)

1.9919434187887219e-05

As you can see, the p-value < the 0.05 significance

4.Write a story describing the statistical significance in the context or the original problem.

5.Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

We can say with 95% confidence that an applicant with a white sounding name is 1.68%-4.73% more likely to get a callback than an apllicant with a black-sounding name. This doesn't indicate if its the DOMINANT factor, as the dataset only provided us with their race. More testing with mroe variables like years or experience is required, but it does seem that race is indeed a factor in callbacks